In [1]:
import scipy.io as sio
import numpy as np
from os import getcwd
from os.path import join 
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.multivariate.pca import PCA
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd 

In [3]:
fname = join(os.getcwd(),'data','data3SS2009.mat')

mat_contents = sio.loadmat(fname)

dataset = mat_contents['dataset']


N, Chno, Nc = dataset.shape
# N: number of samples
# Chno: number of channels
# Nc: number of cases

y = mat_contents['labels'].reshape(Nc)




In [45]:
Ch1 = dataset[:,0,:] # célula de carga: força do shaker
Ch2 = dataset[:,1,:] # acelerômetro: base
Ch3 = dataset[:,2,:] # acelerômetro: 1o andar
Ch4 = dataset[:,3,:] # acelerômetro: 2o andar
Ch5 = dataset[:,4,:] # acelerômetro: 3o andar

Ts = 3.125 * 1e-3 # sampling time
time = (np.linspace(1,N,N) - 1) * Ts


    


In [5]:
ARmodel_2 = AutoReg(Ch2[:,1],30,old_names=False).fit()
ARmodel_3 = AutoReg(Ch3[:,1],30,old_names=False).fit()
ARmodel_4 = AutoReg(Ch4[:,1],30,old_names=False).fit()
ARmodel_5 = AutoReg(Ch5[:,1],30,old_names=False).fit()
vec_2 = ARmodel_2.params
vec_3 = ARmodel_3.params
vec_4 = ARmodel_4.params
vec_5 = ARmodel_5.params
vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))

ARmodel_var = vec_temp

for i in range(1,850):
    ARmodel_2 = AutoReg(Ch2[:,i],30,old_names=False).fit()
    ARmodel_3 = AutoReg(Ch3[:,i],30,old_names=False).fit()
    ARmodel_4 = AutoReg(Ch4[:,i],30,old_names=False).fit()
    ARmodel_5 = AutoReg(Ch5[:,i],30,old_names=False).fit()
    vec_2 = ARmodel_2.params
    vec_3 = ARmodel_3.params
    vec_4 = ARmodel_4.params
    vec_5 = ARmodel_5.params
    vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))
    ARmodel_var = np.vstack([ARmodel_var,vec_temp])

In [ ]:
X = ARmodel_var


In [ ]:
import numpy as np
from scipy import stats
from scipy.stats.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (train_test_split, RepeatedKFold,
RandomizedSearchCV)
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.pipeline import Pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [36]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from scipy.stats import randint
from sklearn.datasets import make_classification
from sklearn.model_selection import ParameterGrid

X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0,random_state=0, shuffle=False)

from sklearn.model_selection import RepeatedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import uniform, randint
from sklearn.ensemble import VotingClassifier

models = {
    "LRG": LinearRegression(),
    "SVR": SVR(),
    "DTR": DecisionTreeRegressor(),
    "KNN": KNeighborsRegressor(),
    "RFR": RandomForestRegressor()
}

param_dists = {
    "LRG": {},
    "SVR": {
        'C': uniform(0.1, 10.0),
        'epsilon': uniform(0.01, 1.0)
    },
     "DTR": {
        'criterion': ['mse', 'friedman_mse',],
        'max_depth': randint(2, 20),
         'splitter' : ["best","freedom"]
    },
    "KNN": {
        'n_neighbors': randint(1, 30),
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    "RFR": {
         'n_estimators': randint(2, 100),
        'max_depth': randint(2, 20),
        'criterion': ['mse', 'mae',],
        'max_features' : stats.uniform()



    }
}

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Perform Randomized Search for each model
for model_name, model in models.items():
    param_dist = param_dists[model_name]
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=100,
        scoring='neg_mean_squared_error',
        cv=cv,
        n_jobs=-1,
        random_state=42
    )
    
    
     random_search.fit(X, y)  

    
    print(f"Best Hyperparameters for {model_name}:", random_search.best_params_)
    print(f"Best Score for {model_name}:", -random_search.best_score_)


C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=100. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Hyperparameters for LRG: {}
Best Score for LRG: 0.07759605785618238
Best Hyperparameters for SVR: {'C': 2.948404943774676, 'epsilon': 0.0468869473545328}
Best Score for SVR: 0.046721744857742276


C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
810 fits failed out of a total of 1500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1315, in fit
    super().fit(
  File "C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 381, in fit
    splitter = SPLITTERS[self.splitter](
KeyError: 'freedom'

  warnings.warn(some_fits_failed_mes

Best Hyperparameters for DTR: {'criterion': 'friedman_mse', 'max_depth': 3, 'splitter': 'best'}
Best Score for DTR: 0.046694611889162775
Best Hyperparameters for KNN: {'n_neighbors': 22, 'p': 2, 'weights': 'distance'}
Best Score for KNN: 0.044905119508557315


C:\Users\lxcas\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


Best Hyperparameters for RFR: {'criterion': 'mse', 'max_depth': 5, 'max_features': 0.942853570557981, 'n_estimators': 72}
Best Score for RFR: 0.03846553120132713
